In [2]:
import pandas as pd
import numpy as np
import glob #lets us 
import os

In [5]:
df_list = []
file_pattern = "*.csv"

file_list = glob.glob(file_pattern)

for file in file_list:
    year = os.path.basename(file)
    year = year.split('.')[0]

    curr = pd.read_csv(file, header = 1)
    curr["Year"] = year
    curr = curr.dropna(axis=1, how='all')
    curr = curr.dropna(subset=['Rk'])


    df_list.append(curr)

combined_df = pd.concat(df_list, ignore_index=True)

In [6]:
combined_df

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G,Year
0,1.0,Milwaukee Bucks*,29.2,56.0,17.0,57,16,10.08,-0.67,9.41,...,20.7,0.201,0.489,12.0,81.6,0.178,Fiserv Forum,549036.0,17711.0,2020
1,2.0,Boston Celtics*,25.3,48.0,24.0,50,22,6.31,-0.47,5.83,...,23.9,0.207,0.509,13.5,77.4,0.215,TD Garden,610864.0,19090.0,2020
2,3.0,Los Angeles Clippers*,27.4,49.0,23.0,50,22,6.44,0.21,6.66,...,23.5,0.233,0.506,12.2,77.6,0.206,STAPLES Center,610176.0,19068.0,2020
3,4.0,Toronto Raptors*,26.6,53.0,19.0,50,22,6.24,-0.26,5.97,...,21.3,0.210,0.502,14.6,76.7,0.202,Scotiabank Arena,633456.0,19796.0,2020
4,5.0,Los Angeles Lakers*,29.5,52.0,19.0,48,23,5.79,0.49,6.28,...,24.5,0.201,0.515,14.1,78.8,0.205,STAPLES Center,588907.0,18997.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,26.0,Phoenix Suns,25.0,24.0,58.0,27,55,-5.63,0.49,-5.14,...,26.0,0.231,0.525,13.1,76.4,0.264,Talking Stick Resort Arena,708639.0,17134.0,2017
296,27.0,Philadelphia 76ers,24.6,28.0,54.0,26,56,-5.70,-0.14,-5.83,...,22.4,0.199,0.512,13.5,75.3,0.234,Wells Fargo Center,710557.0,17331.0,2017
297,28.0,Brooklyn Nets,26.0,20.0,62.0,24,58,-6.73,-0.01,-6.74,...,19.6,0.228,0.513,11.6,76.1,0.212,Barclays Center,632608.0,15429.0,2017
298,29.0,Orlando Magic,25.3,29.0,53.0,24,58,-6.57,-0.04,-6.61,...,21.6,0.184,0.523,11.9,77.4,0.201,Amway Center,727875.0,17753.0,2017
